In [33]:
import io

import boto3
import pandas as pd
import polars as pl

In [25]:
#bucket = 'cdc-moderna-vaccine-distribution/csv/'
bucket = 's3://covid19-lake/cdc-moderna-vaccine-distribution/csv/'
key = 'cdc-moderna-covid-19-vaccine-distribution-by-state.csv'
s3_uri = 's3://covid19-lake/cdc-moderna-vaccine-distribution/csv/cdc-moderna-covid-19-vaccine-distribution-by-state.csv'

In [20]:
#df_down = pl.read_csv(f"s3://{bucket}/{key}")

#pl.read_csv()

In [38]:
# Create a boto3 client to interface with S3
#s3 = boto3.client("s3")

# Set up the S3 client
s3 = boto3.client('s3',
                  aws_access_key_id='',
                  aws_secret_access_key='MoaKXAz/')

s3_resource = boto3.resource('s3')

In [39]:
response = s3.list_buckets()

ClientError: An error occurred (AccessDenied) when calling the ListBuckets operation: Access Denied

In [40]:
trip_bucket = 's3://nyc-tlc'
bucket_resource = s3_resource.Bucket(trip_bucket)

In [41]:
bucket_resource.objects.all()

s3.Bucket.objectsCollection(s3.Bucket(name='s3://nyc-tlc'), s3.ObjectSummary)

In [42]:
def preview_csv_dataset(bucket, key, rows=10):
    data_source = {
            'Bucket': bucket,
            'Key': key
        }
    # Generate the URL to get Key from Bucket
    url = s3.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )

    data = pd.read_csv(url, nrows=rows)
    return data

In [43]:
df = preview_csv_dataset(bucket='nyc-tlc', key='trip data/green_tripdata_2018-02.csv', rows=100)

HTTPError: HTTP Error 403: Forbidden

In [36]:
df.head()

NameError: name 'df' is not defined

In [44]:
bucket_arn = 'arn:aws:s3:::nyc-tlc'
file_key = 'trip data/green_tripdata_2018-01.parquet'
obj = s3.get_object(Bucket=bucket_arn, Key=file_key)

ParamValidationError: Parameter validation failed:
Invalid bucket name "arn:aws:s3:::nyc-tlc": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]*:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-.]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"

In [ ]:
df = pl.read_csv(obj['Body'])

In [45]:
# Define the SQL statement to filter the CSV data
sql_expression = "SELECT * FROM s3object"

# Use SelectObjectContent to filter the CSV data before downloading it
s3_object = s3.select_object_content(
    Bucket=bucket,
    Key=key,
    ExpressionType="SQL",
    Expression=sql_expression,
    InputSerialization={"CSV": {"FileHeaderInfo": "USE"}, "CompressionType": "NONE"},
    OutputSerialization={"CSV": {}},
)
# Create a reusable StringIO object
output = io.StringIO()

# Iterate over the filtered CSV data and write it to the StringIO object
for event in s3_object["Payload"]:
    if "Records" in event:
        records = event["Records"]["Payload"].decode("utf-8")
        output.write(records)

# Rewind the StringIO object to the beginning
output.seek(0)

df = pl.read_csv(output)


ParamValidationError: Parameter validation failed:
Invalid bucket name "s3://covid19-lake/cdc-moderna-vaccine-distribution/csv/": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]*:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-.]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"